In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
from matplotlib import dates as mdate
import json, datetime, pandas
pandas.options.display.float_format = "{:.2f}".format
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import ipywidgets as widgets 
from IPython.display import display, clear_output, Markdown

In [ ]:
def lob_stats_analyser(df):
    outt = widgets.Output()
    stats_checkbox = [widgets.Checkbox(description=stats, value=True,indent=False, disabled=False) for stats in df.columns]
    stats_checkboxes = widgets.HBox(stats_checkbox)
    rolling = widgets.FloatSlider(
             value=50,
             min=1,
             max=200,
             step=1,
             disabled=False,
             description='Rolling:',
            )
    timerange = widgets.IntRangeSlider(
                    value=[0,0],
                    min=0,
                    max=0,
                    step=5,
                    description='Range:',
                    disabled=False,
                    continuous_update=False,
                    orientation='horizontal',
                    readout=True,
                    readout_format='d',
                    layout=dict(width='95%')
                    )
    timerange.max = df.index.max()
    timerange.min = df.index.min()
    timerange.value = [df.index.min(), df.index.max()]

    def plot_time_range(start, end, normalized):
        with outt:
            clear_output()
            timerange.value = [start, end]
            print("Start time:", datetime.datetime.fromtimestamp(start).strftime('%Y-%m-%d %H:%M:%S'),
                    "  End time:", datetime.datetime.fromtimestamp(end).strftime('%Y-%m-%d %H:%M:%S'))
            selected_stats = [checkbox.description for checkbox in stats_checkbox if checkbox.value]
            if normalized:
                data = df[(df.index > start) & 
                    (df.index < end)][selected_stats].rolling(int(rolling.value)).mean().dropna()
                data = data.apply(lambda x: (x-x.min())/(x.max()-x.min()))
            else:
                data = df[(df.index > start) & 
                    (df.index < end)][selected_stats].rolling(int(rolling.value)).mean().dropna()
            plt.rcParams['figure.figsize'] = [15,7]
            plt.rcParams['timezone'] = 'America/Sao_Paulo'

            fig,ax = plt.subplots()
            x = mdate.epoch2num(data.index)
            fmt = mdate.DateFormatter('%H:%M:%S')

            ax.xaxis.set_major_formatter(fmt)
            plt.xticks( rotation=25 )
            plt.plot_date(x, data, linestyle='solid', marker='None')

            plt.legend(data.columns)
            plt.show()
    def on_btn_filter_range_clicked(b):
        plot_time_range(timerange.value[0], timerange.value[1], False)
    def on_btn_filter_range_norm_clicked(b):
        plot_time_range(timerange.value[0], timerange.value[1], True)
    btn_filter_range = widgets.Button(description='Apply Filter',button_style='danger', disabled=False)
    btn_filter_range.on_click(on_btn_filter_range_clicked)
    btn_filter_range_norm = widgets.Button(description='Apply Filter w/ Norm',button_style='warning', disabled=False)
    btn_filter_range_norm.on_click(on_btn_filter_range_norm_clicked)
    btns = widgets.HBox([btn_filter_range, btn_filter_range_norm])

    return widgets.VBox([stats_checkboxes, rolling, timerange,btns, outt])

In [ ]:
df = pandas.read_csv('bpac11_bear_market_20210308.csv',sep=';')
df.set_index('timestamp', inplace=True)
df.head()

In [ ]:
lob_stats_analyser(df)